In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch_geometric as pyg
import torch.utils

class InteractionNetwork(pyg.nn.MessagePassing):
    def __init__(self, hidden_size, layers=3):
        super().__init__()
        self.lin_edge = torchvision.ops.MLP(hidden_size*3, hidden_size, layers)
        torch.nn.MLP()